In [1]:
import os, json
import pandas as pd
import numpy as np

import spotipy
import spotipy.util as util

In [3]:
# set API keys
apikeys = json.load(open("../data/api-keys.json"))
os.environ["SPOTIPY_CLIENT_ID"]     = apikeys["spotipy-client-id"]
os.environ["SPOTIPY_CLIENT_SECRET"] = apikeys["spotipy-client-secret"]
os.environ["SPOTIPY_REDIRECT_URI"]  = apikeys["redirect-url"]

# set my user_id
user_id = '129874447'

In [4]:
# connect to spotify
token = util.prompt_for_user_token(user_id, scope = 'user-library-read')
sp = spotipy.Spotify(auth = token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=8ab10b4296bd4266b959f2caee8dd7a4&response_type=code&redirect_uri=http%3A%2F%2Fben-tanen.com&scope=user-library-read in your browser


Enter the URL you were redirected to: http://ben-tanen.com/?code=AQAe3BhYyY-uwvodCHehi7lSXFaHYtJaKD0LZ998XaZsOwhe6vH8FZ8_gTQFxBUcaCc59J0iyZT6FJhug_lHHxUg4A5G_ZoZbYx_2iaQMPaeiwq4Sd0AeHcpEkss3pK1qELMsE5OXe9Hzmb8s0LXUGtCfsUxog8gqhE03t-byWakDtkVDz39zdFkFw




### Define helper functions

In [24]:
### function to get the current user's saved tracks (track name, artist, id)
def get_saved_artists(limit = 50, offset = 0):
    saved_artists = [ ]
    
    # get initial list of tracks to determine length
    saved_tracks_obj = sp.current_user_saved_tracks(limit = limit, offset = offset)
    num_saved_tracks = saved_tracks_obj['total']
    
    # loop through to get all saved tracked
    while (offset < num_saved_tracks):
        saved_tracks_obj = sp.current_user_saved_tracks(limit = limit, offset = offset)
        
        # add track information to running list
        for track_obj in saved_tracks_obj['items']:
            for artist in track_obj['track']['artists']:
                saved_artists.append({
                    'artist': artist['name'],
                    'artist_id': artist['id']
                })
    
        offset += limit
        
    return saved_artists

def get_artist_genres(artist_ids):
    print(artist_ids)

In [91]:
artists = pd.DataFrame(get_saved_artists())

In [92]:
artists_uniq = pd.DataFrame(artists.groupby(['artist', 'artist_id']).size(), columns = ["count"]).reset_index()
artist_ids = artists_uniq['artist_id']

In [93]:
artist_ids

0      17lzZA2AlOHwCwFALHttmp
1      5uhqkMm8dyQvX83kl4Znq0
2      08Uextujt6ZT2iQmSYAJfH
3      6FfsOtDdMCFO5Qo0H8rwPa
4      2XxgHIPXNgHGbvtBBGZrKm
                ...          
889    7sOR7gk6XUlGnxj3p9F54k
890    3ztRX1UoIOsFqpD7dB6R8O
891    5xeBMeW0YzWIXSVzAxhM8O
892    4h7DUL1L3RrCzquDp8xQXY
893    7xUZ4069zcyBM4Bn10NQ1c
Name: artist_id, Length: 894, dtype: object

In [94]:
artist_genres = [ ]
    
# iterate through track_ids in groups of 50
for ix in range(0,len(artist_ids), 50):
    artist_genres += [{'id': artist_obj['id'],
                       'name': artist_obj['name'],
                       'genres': '|'.join(artist_obj['genres'])}
                      for artist_obj in sp.artists(artist_ids[ix:ix+50])['artists']]

In [102]:
artists_uniq = artists_uniq.merge(pd.DataFrame(artist_genres), \
                                  how = 'left', left_on = 'artist_id', right_on = 'id').drop(['id', 'name'], axis = 1)

In [103]:
artists_uniq

,artist,artist_id,count,genres
0,2 Chainz,17lzZA2AlOHwCwFALHttmp,1,atl hip hop|gangster rap|hip hop|pop rap|rap|s...
1,99 Neighbors,5uhqkMm8dyQvX83kl4Znq0,2,vermont indie
2,AHI,08Uextujt6ZT2iQmSYAJfH,2,canadian folk
3,ARMAND MARGJEKA,6FfsOtDdMCFO5Qo0H8rwPa,2,
4,ASL,2XxgHIPXNgHGbvtBBGZrKm,1,
...,...,...,...,...
889,half•alive,7sOR7gk6XUlGnxj3p9F54k,2,indie pop|modern alternative rock|modern rock|...
890,illuminati hotties,3ztRX1UoIOsFqpD7dB6R8O,1,indie punk|indie rock|small room
891,of Montreal,5xeBMeW0YzWIXSVzAxhM8O,1,alternative dance|alternative rock|anti-folk|a...
892,yMusic,4h7DUL1L3RrCzquDp8xQXY,1,modern performance


In [104]:
artists_uniq.to_csv('../data/2020.07.25 - artist genres.csv', index = False)

In [108]:
pd.DataFrame(json.load(open("../data/2020.07.25 - every-noise-at-once-scrape.txt"))).to_csv("../data/2020.07.25 - every-noise-at-once-scrape.csv", index = False)